# 2. Text Data Preprocessing

In [3]:
import torch 
import torch.nn as nn
from torch.utils import data

import re
import collections

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
from d2l import torch as d2l

Typical **preprocessing pipelines** of text data execute the following steps:

1. Load text as strings into memory.
2. Split the strings into tokens.
3. Build a vocabulary dictionary to associate each word with a numerical index.
4. Convert the text into sequences of numerical indices.

## Load Data

For simplicity, we ignore **punctuation** and **capitalization** when loading the raw text:

In [4]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine(): 
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'Total number of lines: {len(lines)}')
print(lines[0])
print(lines[10])

Total number of lines: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## Tokenization

The following **tokenizer** function takes **lines (a list)** of text as input (each line is a sequence of words) and **outputs** a list of **token lists**:

In [5]:
def tokenize(lines, token='word'):
    if token=='word':
        return [line.split() for line in lines]
    elif token=='char':
        return [list(line) for line in lines]
    else:
        print('Error: wrong type of tokens.')

In [7]:
tokens = tokenize(lines)
tokens[0]

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']

## Vocabulary

Now we need to construct a **vocabulary** that maps the tokens into **numerical indices** starting from 0.

First, we need to create a **corpus** that contains all the **unique tokens** in the text data. Then, we **rank** the tokens according to their **frequencies** of appearance and map them to numerical indices. Tokens with extremely low frequencies will be removed for simplicity.

Tokens that are **new to the corpus** will be mapped to a special label **\<unk\>**.

In [12]:
def count_corpus(tokens): 
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [14]:
class Vocab:  
    
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
            
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self): 
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

In [16]:
vocab = Vocab(tokens)
list(vocab.token_to_idx.items())[:10]

[('<unk>', 0),
 ('the', 1),
 ('i', 2),
 ('and', 3),
 ('of', 4),
 ('a', 5),
 ('to', 6),
 ('was', 7),
 ('in', 8),
 ('that', 9)]

In [20]:
print('Text:', tokens[0])
print('Indices:', vocab[tokens[0]])

Text: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
Indices: [1, 19, 50, 40, 2183, 2184, 400]


## All in One Step

In [39]:
def load_corpus_time_machine(max_tokens=-1):  
    lines = read_time_machine()
    tokens = tokenize(lines, 'word')
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

In [40]:
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(32775, 4580)